# Notebook to process using Pandas
 * Read input files
 * Join both the input files by counter_party
 * Carry out aggregation by maximum of ratings
 * Carry out aggregation by sum of values
 * Split the dataframe based on status
 * Finally merge all the aggregated dataframes to generate the report.

In [53]:
import pandas as pd
import numpy as np
from functools import reduce
from IPython.display import HTML

In [54]:
ds1 = pd.read_csv('input/dataset1.csv')

In [55]:
def display_df_head(df):
    return HTML(df.head().to_html(index=False))

In [56]:
display_df_head(ds1)

invoice_id,legal_entity,counter_party,rating,status,value
1,L1,C1,1,ARAP,10
2,L2,C2,2,ARAP,20
3,L3,C3,4,ACCR,30
4,L1,C4,6,ARAP,40
5,L2,C5,4,ACCR,50


In [57]:
ds2 = pd.read_csv('input/dataset2.csv')

In [58]:
merged_df = pd.merge(ds1,ds2, on = ['counter_party'])

In [59]:
display_df_head(merged_df)

invoice_id,legal_entity,counter_party,rating,status,value,tier
1,L1,C1,1,ARAP,10,1
7,L1,C1,2,ARAP,10,1
13,L1,C1,3,ARAP,20,1
2,L2,C2,2,ARAP,20,2
8,L2,C2,3,ACCR,40,2


In [60]:
columns_to_group = ['legal_entity', 'counter_party', 'tier']
columns_to_group_with_sum_values = columns_to_group + ['sum_values']
columns_to_group_with_status = columns_to_group + ['status']

In [61]:
df_max_rating = merged_df.groupby(columns_to_group, as_index=False).agg(max_rating=('rating','max'))

In [62]:
display_df_head(df_max_rating)

legal_entity,counter_party,tier,max_rating
L1,C1,1,3
L1,C3,3,6
L1,C4,4,6
L2,C2,2,3
L2,C3,3,2


In [63]:
df_unique_record_counts = merged_df.groupby(columns_to_group, as_index=False).agg(unique_record_counts=('rating','count'))

In [64]:
display_df_head(df_unique_record_counts)

legal_entity,counter_party,tier,unique_record_counts
L1,C1,1,3
L1,C3,3,1
L1,C4,4,2
L2,C2,2,2
L2,C3,3,1


In [65]:
df_sum_values = merged_df.groupby(columns_to_group_with_status, as_index=False).agg(sum_values = ('value','sum'))

In [66]:
display_df_head(df_sum_values)

legal_entity,counter_party,tier,status,sum_values
L1,C1,1,ARAP,40
L1,C3,3,ARAP,5
L1,C4,4,ACCR,100
L1,C4,4,ARAP,40
L2,C2,2,ACCR,40


In [67]:
df_sum_values_ARAP = df_sum_values[(df_sum_values['status'] == 'ARAP')][columns_to_group_with_sum_values]
df_sum_values_ARAP = df_sum_values_ARAP.rename(columns={'sum_values': 'sum(values where status=ARAP)'})

In [68]:
display_df_head(df_sum_values_ARAP)

legal_entity,counter_party,tier,sum(values where status=ARAP)
L1,C1,1,40
L1,C3,3,5
L1,C4,4,40
L2,C2,2,20
L2,C5,5,1000


In [69]:
df_sum_values_ACCR = df_sum_values[(df_sum_values['status'] == 'ACCR')][columns_to_group_with_sum_values]
df_sum_values_ACCR = df_sum_values_ACCR.rename(columns={'sum_values': 'sum(values where status=ACCR)'})

In [70]:
display_df_head(df_sum_values_ACCR)

legal_entity,counter_party,tier,sum(values where status=ACCR)
L1,C4,4,100
L2,C2,2,40
L2,C3,3,52
L2,C5,5,115
L3,C3,3,145


In [71]:
consolidated_df_list = [df_max_rating,df_unique_record_counts,df_sum_values_ARAP,df_sum_values_ACCR]

In [72]:
df_report = reduce(lambda left, right: pd.merge(left, right,
                                                on=columns_to_group, how='outer'), consolidated_df_list).fillna(0)

In [73]:
display_df_head(df_report)

legal_entity,counter_party,tier,max_rating,unique_record_counts,sum(values where status=ARAP),sum(values where status=ACCR)
L1,C1,1,3,3,40.0,0.0
L1,C3,3,6,1,5.0,0.0
L1,C4,4,6,2,40.0,100.0
L2,C2,2,3,2,20.0,40.0
L2,C3,3,2,1,0.0,52.0


In [74]:
df_report.to_csv('output/report_pandas.csv')